### CHALLENGE
- Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits
- We will be provided a company name and their primary website

In [15]:
# imports required libraries and modules
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [16]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [17]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
company_name = "Maersk"
company_website_url = "https://www.maersk.com"

client_website = Website(company_website_url)
print(client_website.links)
client_website.links

['/digital-services', '/support/get-started', '/portaluser/register/', '/logistics-explained', '/east-west-network', '/insights/logistics-trend-map', '/industry-sectors/fmcg', '/industry-sectors/retail', '/industry-sectors/lifestyle', '/industry-sectors/chemicals', '/industry-sectors/cold-chain-perishables', '/industry-sectors/automotive', '/industry-sectors/pharma-healthcare-logistics', '/industry-sectors/technology', '/insights/filter', '/news', '/campaigns/global-tariffs']


['/digital-services',
 '/support/get-started',
 '/portaluser/register/',
 '/logistics-explained',
 '/east-west-network',
 '/insights/logistics-trend-map',
 '/industry-sectors/fmcg',
 '/industry-sectors/retail',
 '/industry-sectors/lifestyle',
 '/industry-sectors/chemicals',
 '/industry-sectors/cold-chain-perishables',
 '/industry-sectors/automotive',
 '/industry-sectors/pharma-healthcare-logistics',
 '/industry-sectors/technology',
 '/insights/filter',
 '/news',
 '/campaigns/global-tariffs']

## First step: Have GPT-4o-mini figure out which links are relevant
### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [20]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [21]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [22]:
print(get_links_user_prompt(client_website))

Here is the list of links on the website of https://www.maersk.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/digital-services
/support/get-started
/portaluser/register/
/logistics-explained
/east-west-network
/insights/logistics-trend-map
/industry-sectors/fmcg
/industry-sectors/retail
/industry-sectors/lifestyle
/industry-sectors/chemicals
/industry-sectors/cold-chain-perishables
/industry-sectors/automotive
/industry-sectors/pharma-healthcare-logistics
/industry-sectors/technology
/insights/filter
/news
/campaigns/global-tariffs


In [24]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace.
huggingface = Website("https://huggingface.co")
huggingface.links

In [25]:
get_links(company_website_url)

{'links': [{'type': 'about page',
   'url': 'https://www.maersk.com/digital-services'},
  {'type': 'logistics explained',
   'url': 'https://www.maersk.com/logistics-explained'},
  {'type': 'industry sectors - FMCG',
   'url': 'https://www.maersk.com/industry-sectors/fmcg'},
  {'type': 'industry sectors - retail',
   'url': 'https://www.maersk.com/industry-sectors/retail'},
  {'type': 'industry sectors - lifestyle',
   'url': 'https://www.maersk.com/industry-sectors/lifestyle'},
  {'type': 'industry sectors - chemicals',
   'url': 'https://www.maersk.com/industry-sectors/chemicals'},
  {'type': 'industry sectors - cold chain perishables',
   'url': 'https://www.maersk.com/industry-sectors/cold-chain-perishables'},
  {'type': 'industry sectors - automotive',
   'url': 'https://www.maersk.com/industry-sectors/automotive'},
  {'type': 'industry sectors - pharma healthcare logistics',
   'url': 'https://www.maersk.com/industry-sectors/pharma-healthcare-logistics'},
  {'type': 'industry sec

### Second step: make the brochure!
Assemble all the details into another prompt to GPT4-o

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details(company_website_url))

Found links: {'links': [{'type': 'digital services', 'url': 'https://www.maersk.com/digital-services'}, {'type': 'logistics explained', 'url': 'https://www.maersk.com/logistics-explained'}, {'type': 'industry sectors FMCG', 'url': 'https://www.maersk.com/industry-sectors/fmcg'}, {'type': 'industry sectors retail', 'url': 'https://www.maersk.com/industry-sectors/retail'}, {'type': 'industry sectors lifestyle', 'url': 'https://www.maersk.com/industry-sectors/lifestyle'}, {'type': 'industry sectors chemicals', 'url': 'https://www.maersk.com/industry-sectors/chemicals'}, {'type': 'industry sectors cold chain perishables', 'url': 'https://www.maersk.com/industry-sectors/cold-chain-perishables'}, {'type': 'industry sectors automotive', 'url': 'https://www.maersk.com/industry-sectors/automotive'}, {'type': 'industry sectors pharma healthcare logistics', 'url': 'https://www.maersk.com/industry-sectors/pharma-healthcare-logistics'}, {'type': 'industry sectors technology', 'url': 'https://www.ma

In [28]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt(company_name, company_website_url)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.maersk.com/digital-services'}, {'type': 'logistics explained', 'url': 'https://www.maersk.com/logistics-explained'}, {'type': 'news page', 'url': 'https://www.maersk.com/news'}]}


"You are looking at a company called: Maersk\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMaersk | Integrated Container Logistics & Supply Chain Services\nWebpage Contents:\nUnlock the full potential of your supply chain\nThis is logistics the simple way: Convenient and connected across ocean, land, and air – and with all your needs covered from end to end.\nAccess now\nB/L, container number or parcel\nThis is a required field.\nTrack\nFrom\nOrigin is incorrect\nTo\nDestination is incorrect\nSearch\nSearch for an office\nThis is a required field\nFind\nHow to get started\nWhether you need to find prices, place a booking, submit documents, or track your cargo, our step-by-step guides will help you find your way around.\nGet started\nReady to book?\nSign up to book, manage and pay your shipments online, and access a suite of products and services designed to 

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [32]:
create_brochure(company_name, company_website_url)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.maersk.com/digital-services'}, {'type': 'about page', 'url': 'https://www.maersk.com/logistics-explained'}, {'type': 'about page', 'url': 'https://www.maersk.com/insights/logistics-trend-map'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/fmcg'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/retail'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/lifestyle'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/chemicals'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/cold-chain-perishables'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/automotive'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/pharma-healthcare-logistics'}, {'type': 'about page', 'url': 'https://www.maersk.com/industry-sectors/technology'}, {'type': 'news page', 'url': 'https://www.

```markdown
# Maersk: Transforming Logistics with Integrated Solutions

## Unlock Your Supply Chain Potential
At Maersk, we simplify logistics across ocean, land, and air, offering comprehensive end-to-end services tailored to meet your logistical needs. With a focus on transparency and efficiency, our innovative platform provides solutions that empower businesses to thrive, regardless of industry or scale.

## Our Services
We provide a diverse range of logistics and supply chain management services, including:
- **Transportation**: Seamlessly move your goods via our extensive global network.
- **Storage & Protection**: Secure warehousing solutions that cater to your unique requirements.
- **Logistics Management**: End-to-end support for managing your shipments effectively.
  
Explore our **East-West Network**, setting new standards in reliability and service quality, ensuring world-class productivity through strategically located hubs.

## Who We Serve
Maersk supports a variety of industries, including:
- Fast-Moving Consumer Goods (FMCG)
- Retail
- Fashion & Lifestyle
- Pharmaceuticals & Healthcare
- Automotive
- Technology

## Company Culture
At Maersk, we believe in creating an inclusive environment where every employee is valued and encouraged to grow. We embrace diversity, foster innovation, and focus on sustainability across our operations. Our culture thrives on collaboration and continuous improvement, enabling us to respond rapidly and effectively to our customers' logistics needs.

## Career Opportunities
Join the Maersk team to make a difference in the logistics industry. We seek passionate self-starters ready to tackle challenges and innovate. Whether you are in operations, customer service, or technological development, your contributions will resonate across our global network.

- **Job Roles**: Logistics Experts, Operations Managers, Supply Chain Analysts, Customer Service Representatives
- **Benefits**: Competitive salaries, career development programs, and a supportive work environment.

## Engage with Us
- **Website**: [maersk.com](https://www.maersk.com)
- **Register Now** to experience our seamless booking and shipment management services.
- **Stay Informed**: Access the latest industry insights, trends, and updates through our dedicated news portal.

## Why Choose Maersk?
- **All-in-One Platform**: Handle your logistics workflow from booking to delivery seamlessly.
- **Transparent Pricing**: Avoid hidden costs with clear pricing models at the point of booking.
- **Global Reach**: Access over 500 ports in 130+ countries, ensuring maximum connectivity.
- **Real-Time Tracking**: Gain visibility throughout the entire shipping process, so you always know where your cargo stands.

Join us at Maersk, where logistics is not just a service, but a pathway to unlocking your business's potential.
```

In [33]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure(company_name, company_website_url)

Found links: {'links': [{'type': 'digital services page', 'url': 'https://www.maersk.com/digital-services'}, {'type': 'logistics explained page', 'url': 'https://www.maersk.com/logistics-explained'}, {'type': 'industry sectors page', 'url': 'https://www.maersk.com/industry-sectors'}, {'type': 'news page', 'url': 'https://www.maersk.com/news'}]}


# Maersk Brochure

## Welcome to Maersk: Integrated Container Logistics & Supply Chain Services

Unlock the full potential of your supply chain with Maersk, your partner for integrated logistics solutions. We simplify logistics across ocean, land, and air, ensuring your business's needs are met from end to end.

---

### Why Choose Maersk?

- **End-to-End Logistics**: Handle your logistics workflow—from booking to delivery—through a single platform.
- **Transparent Pricing**: Experience clear, upfront pricing at the time of booking, eliminating guesswork.
- **Global Coverage**: Access over 500 ports in 130+ countries and benefit from air freight services across 70,000 trade lanes in 90+ markets.

### Our Services

- **Transport**: Efficient movement of goods across various modes of transportation.
- **Storage**: Reliable storage solutions tailored to your commodity needs.
- **Logistics Management**: Comprehensive management and support for your logistics processes.
- **Clear & Protect**: Ensuring your goods are securely handled and protected.

### Tailored Solutions for Every Industry

Maersk supports a wide range of industry sectors, including:

- FMCG
- Retail
- Fashion & Lifestyle
- Chemicals
- Perishables
- Automotive
- Pharma & Healthcare
- Technology

---

### Company Culture

At Maersk, we foster a values-driven culture that prioritizes innovation and collaboration. We believe in empowering our employees to be logistics experts through continuous learning and development opportunities. Our dynamic work environment encourages teamwork and personal initiative, leading to success for both our clients and our people.

---

### Customers: Your Success is Our Business

Whether you are a small startup or a large enterprise, Maersk provides logistics solutions that ensure your business can thrive in a competitive landscape. We are dedicated to delivering reliable, efficient, and customized services that cater to the unique needs of our customers.

---

### Careers at Maersk

Join our diverse team and become part of a global leader in logistics. We are always on the lookout for talent that thrives in a fast-paced, innovative environment. Explore a variety of career paths at Maersk and help us shape the future of logistics.

- **Growth Opportunities**: Develop and advance your career with training programs and workshops.
- **Inclusive Environment**: Work in a diverse setting that values the contributions of every employee.
- **Impactful Work**: Be part of a company that drives change in the logistics and supply chain industry.

---

For more detailed information on our services or to start your logistics journey with us, [visit our website](#) or [register now](#).

**Maersk: Simplifying Logistics, Empowering Business.**